# Fashion MNIST Deep Learning Project

## 1. Problem Definition
## 2. Load Data
## 3. Train/Validation/Test Split
## 4. Preprocessing (Split first, then normalize)
## 5. Build CNN Model
## 6. Train With Early Stopping
## 7. Evaluate on Test
## 8. Conclusions